In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = [25, 8]
plt.style.use("fivethirtyeight")

pd.options.display.max_rows=1000

import warnings
warnings.simplefilter('ignore')

In [ ]:
# train-test data set
train=pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv',index_col='row_id', parse_dates=['time'])
train.head()

In [ ]:
test=pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col='row_id', parse_dates=['time'])
test.head()

In [ ]:
sns.histplot(train.congestion)

In [ ]:
train = train[(train.time.dt.month != 5) | (train.time.dt.day != 27)]

# July 4
train= train[(train.time.dt.month != 7) | (train.time.dt.day != 4)]

# Labor Day
train = train[(train.time.dt.month != 9) | (train.time.dt.day != 2)]
train.shape

In [ ]:
def place_dir(df):
    return df.apply(lambda row: f"{row.x}-{row.y}-{row.direction}", axis=1).values.reshape([-1, 1])

for df in [train, test]:
    df['place_dir'] = place_dir(df)
    

In [ ]:
train.head()

In [ ]:
for df in [train, test]:
    df['weekday'] = df.time.dt.weekday
    df['hour'] = df.time.dt.hour
    df['minute'] = df.time.dt.minute
    df['weekend'] = df.time.dt.dayofweek >=5
    #df['day'] = df.time.dt.day
    #df['month'] = df.time.dt.month

In [ ]:
medians = train.groupby(['x', 'y', 'direction', 'weekday', 'hour', 'minute','place_dir','weekend']).congestion.mean().astype(int)
medians


In [ ]:
sub = test.merge(medians, 
                 left_on=['x', 'y', 'direction', 'weekday', 'hour', 'minute','place_dir','weekend'],
                 right_index=True)[['congestion']]
sub.reset_index(inplace=True)
sub.to_csv('submission_no_machine_learning3.csv', index=False)
sub

In [ ]:
print("train data shape :",train.shape)
print("test data shape :",test.shape)

In [ ]:
train.describe().T

In [ ]:
test.describe().T

In [ ]:
# Missing values
print("Null values in train :",train.isnull().sum().sum())
print("Null values in test :",test.isnull().sum().sum())

In [ ]:
target='congestion'
col='x'
plt.figure()
sns.distplot(train[col][train[target] <50], color='b', label='1') # , hist=False
sns.distplot(train[col][train[target] >50], color='g', label='0') # , hist=False
plt.legend(loc='best')
plt.show()

In [ ]:
col='y'
plt.figure()
sns.distplot(train[col][train[target] <50], color='b', label='1') # , hist=False
sns.distplot(train[col][train[target] >50], color='g', label='0') # , hist=False
plt.legend(loc='best')
plt.show()

In [ ]:
train['direction']=train['direction'].astype('category').cat.codes

In [ ]:
col='direction'
plt.figure()
sns.distplot(train[col][train[target] <50], color='b', label='1') # , hist=False
sns.distplot(train[col][train[target] >50], color='g', label='0') # , hist=False
plt.legend(loc='best')
plt.show()

In [ ]:
train['time'] = pd.to_datetime(train['time'])

In [ ]:
train['month'] = train['time'].dt.strftime('%b')
train['year'] = [d.year for d in train.time]
train['month'] = [d.strftime('%b') for d in train.time]
years = train['year'].unique()
plt.figure(figsize=(15,6))
sns.boxplot(x='month', y='congestion', data=train).set_title("Multi Month-wise Box Plot")
plt.show()

In [ ]:
print('Thanks You!')